# 

In [2]:
#SOURCES
dataset = "https://datos.gob.mx/busca/dataset/indicadores-de-pobreza-pobreza-por-ingresos-rezago-social-y-gini-a-nivel-municipal1990-200-2010"
head_description = "https://www.coneval.org.mx/Informes/Pobreza/Datos_abiertos/Indicadores_municipales/Indicadores_municipales_sabana_DIC.txt"

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter

In [4]:
# Start by creating a dataframe from the dataset
file = 'Indicadores_municipales_sabana_DA.csv'
df = pd.read_csv(file, encoding='latin-1')
var_Ytarget = df["ic_segsoc"] #target variable
var_Ytarget.describe() # see information of target

count    2456.000000
mean       78.753363
std        15.354490
min        24.201883
25%        70.657546
50%        82.849633
75%        90.958454
max        99.283028
Name: ic_segsoc, dtype: float64

In [5]:
# See number of empty spaces
null_data = df.isna().sum().sum()
print("null values =>", null_data)

null values => 305


In [6]:
# Cleaning the empty spaces
df.bfill(inplace=True) #backwards
df.ffill(inplace=True) #forwards
null_data = df.isna().sum().sum()
print("null values =>", null_data)

null values => 0


In [7]:
#Seeing the dimentionality of our dataset
rows = df.shape[0] 
columns = df.shape[1]
print(f"rows => {rows} x columns => {columns}")

rows => 2456 x columns => 139


,ent,nom_ent,mun,clave_mun,nom_mun,pobtot_ajustada,pobreza,pobreza_e,pobreza_m,vul_car,...,pobreza_alim_10,pobreza_cap_90,pobreza_cap_00,pobreza_cap_10,pobreza_patrim_90,pobreza_patrim_00,pobreza_patrim_10,gini_90,gini_00,gini_10
0,1,Aguascalientes,1,1001,Aguascalientes,794304,30.531104,2.264478,28.266627,27.983320,...,11.805700,20.4,12.7,18.474600,43.4,33.7,41.900398,0.473,0.425,0.422628
1,1,Aguascalientes,2,1002,Asientos,48592,67.111172,8.040704,59.070468,22.439389,...,21.993299,39.9,29.0,30.980801,64.2,48.9,59.175800,0.379,0.533,0.343879
2,1,Aguascalientes,3,1003,Calvillo,53104,61.360527,7.241238,54.119289,29.428583,...,19.266800,39.5,33.1,28.259199,63.9,57.9,56.504902,0.414,0.465,0.386781
3,1,Aguascalientes,4,1004,Cosío,14101,52.800458,4.769001,48.031458,27.128568,...,14.303200,35.2,21.0,22.386101,59.7,40.1,51.164501,0.392,0.541,0.344984
4,1,Aguascalientes,5,1005,Jesús María,101379,45.338512,6.084037,39.254475,26.262912,...,15.085100,36.6,22.6,22.139999,60.6,42.2,45.703899,0.391,0.469,0.458083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,32,Zacatecas,54,32054,Villa Hidalgo,21016,74.848837,12.301183,62.547654,19.229856,...,30.055300,51.8,54.8,41.368999,73.5,70.9,70.859596,0.403,0.589,0.342037
2452,32,Zacatecas,55,32055,Villanueva,27385,65.450191,10.203506,55.246687,23.623556,...,13.138800,34.2,25.9,20.563601,57.8,44.1,46.659199,0.422,0.463,0.362527
2453,32,Zacatecas,56,32056,Zacatecas,117528,29.541959,3.535624,26.006335,16.644262,...,7.164800,15.7,20.7,12.115300,36.6,41.8,32.302700,0.528,0.498,0.436339
2454,32,Zacatecas,57,32057,Trancoso,20456,78.374962,14.607016,63.767946,13.750759,...,21.285900,36.2,36.4,30.037100,60.5,54.7,57.394501,0.380,0.483,0.365307


In [8]:
# Filtering the data, by dropping all data related with years, due to our prediction is not a valuable info
years = ["90","00","05","10","_00", "_05", "_10", "_90"]
# Filtra las columnas que no terminan con los sufijos de años 
df = df[[col for col in df.columns if not col.endswith(tuple(years))]]


In [9]:
# Based on the selected target create a new column and fill with 0/1
df['secure_live'] = (var_Ytarget <= 50).astype(float) 
i_secure_live = df.columns.get_loc('secure_live')
print(i_secure_live)

#Comparing the changes made with the target variable
df_compare = df[['ic_segsoc', 'secure_live']]
print(df_compare)
# Deleting the old target column.
df = df.drop(columns='ic_segsoc')
df

56
      ic_segsoc  secure_live
0     41.799885          1.0
1     78.003570          0.0
2     80.051980          0.0
3     65.831374          0.0
4     52.616992          0.0
...         ...          ...
2451  76.550988          0.0
2452  74.542926          0.0
2453  32.666426          1.0
2454  83.235286          0.0
2455  76.211864          0.0

[2456 rows x 2 columns]


C:\Users\ricar\AppData\Local\Temp\ipykernel_14244\2562016240.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['secure_live'] = (var_Ytarget <= 50).astype(float)


,ent,nom_ent,mun,clave_mun,nom_mun,pobtot_ajustada,pobreza,pobreza_e,pobreza_m,vul_car,...,cpic_asalud,cpic_segsoc,cpic_cv,cpic_sbv,cpic_ali,cpcarencias,cpcarencias3,cpplb,cpplbm,secure_live
0,1,Aguascalientes,1,1001,Aguascalientes,794304,30.531104,2.264478,28.266627,27.983320,...,2.327308,2.039859,2.642070,2.785471,2.350738,1.818455,3.311549,1.495813,1.814550,1.0
1,1,Aguascalientes,2,1002,Asientos,48592,67.111172,8.040704,59.070468,22.439389,...,2.551830,1.946811,3.043817,3.020979,2.667471,1.887565,3.382552,1.790992,1.945454,0.0
2,1,Aguascalientes,3,1003,Calvillo,53104,61.360527,7.241238,54.119289,29.428583,...,2.438658,1.923021,3.081348,3.001547,2.847499,1.850954,3.317859,1.818049,1.976999,0.0
3,1,Aguascalientes,4,1004,Cosío,14101,52.800458,4.769001,48.031458,27.128568,...,2.495737,1.796453,2.932659,2.960497,2.788465,1.713273,3.355825,1.541247,1.728584,0.0
4,1,Aguascalientes,5,1005,Jesús María,101379,45.338512,6.084037,39.254475,26.262912,...,2.390123,2.023067,2.764593,3.375908,2.564959,1.852616,3.320841,1.652841,1.993144,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,32,Zacatecas,54,32054,Villa Hidalgo,21016,74.848837,12.301183,62.547654,19.229856,...,2.730348,2.177080,3.615010,3.326316,2.729491,2.091074,3.403502,2.029167,2.120813,0.0
2452,32,Zacatecas,55,32055,Villanueva,27385,65.450191,10.203506,55.246687,23.623556,...,2.522609,2.295629,3.713783,3.069458,3.148968,2.185606,3.334456,2.074749,2.210380,0.0
2453,32,Zacatecas,56,32056,Zacatecas,117528,29.541959,3.535624,26.006335,16.644262,...,2.562583,2.132257,3.146202,2.999603,2.407763,1.877738,3.363973,1.536344,2.010299,1.0
2454,32,Zacatecas,57,32057,Trancoso,20456,78.374962,14.607016,63.767946,13.750759,...,2.944920,2.048130,3.336435,2.790056,2.880368,1.979787,3.387613,1.912216,2.069847,0.0


In [10]:
#Deleting not relevant columns with numerical and string type
df = df.drop(columns=df.columns[:5],axis=1)
df 

,pobtot_ajustada,pobreza,pobreza_e,pobreza_m,vul_car,vul_ing,npnv,ic_rezedu,ic_asalud,ic_cv,...,cpic_asalud,cpic_segsoc,cpic_cv,cpic_sbv,cpic_ali,cpcarencias,cpcarencias3,cpplb,cpplbm,secure_live
0,794304,30.531104,2.264478,28.266627,27.983320,8.419106,33.066469,14.970553,24.034493,4.721187,...,2.327308,2.039859,2.642070,2.785471,2.350738,1.818455,3.311549,1.495813,1.814550,1.0
1,48592,67.111172,8.040704,59.070468,22.439389,5.557604,4.891835,21.222712,15.514032,11.062247,...,2.551830,1.946811,3.043817,3.020979,2.667471,1.887565,3.382552,1.790992,1.945454,0.0
2,53104,61.360527,7.241238,54.119289,29.428583,2.921336,6.289554,27.361207,20.812551,9.880071,...,2.438658,1.923021,3.081348,3.001547,2.847499,1.850954,3.317859,1.818049,1.976999,0.0
3,14101,52.800458,4.769001,48.031458,27.128568,7.709276,12.361698,20.889023,14.071657,11.443449,...,2.495737,1.796453,2.932659,2.960497,2.788465,1.713273,3.355825,1.541247,1.728584,0.0
4,101379,45.338512,6.084037,39.254475,26.262912,8.279864,20.118712,20.578144,16.567818,12.936542,...,2.390123,2.023067,2.764593,3.375908,2.564959,1.852616,3.320841,1.652841,1.993144,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,21016,74.848837,12.301183,62.547654,19.229856,3.177689,2.743618,27.350040,36.056322,5.679670,...,2.730348,2.177080,3.615010,3.326316,2.729491,2.091074,3.403502,2.029167,2.120813,0.0
2452,27385,65.450191,10.203506,55.246687,23.623556,5.007426,5.918827,29.914879,53.313420,2.857021,...,2.522609,2.295629,3.713783,3.069458,3.148968,2.185606,3.334456,2.074749,2.210380,0.0
2453,117528,29.541959,3.535624,26.006335,16.644262,8.828019,44.985759,11.936088,18.316528,2.417258,...,2.562583,2.132257,3.146202,2.999603,2.407763,1.877738,3.363973,1.536344,2.010299,1.0
2454,20456,78.374962,14.607016,63.767946,13.750759,4.440331,3.433948,26.649950,11.769479,10.705211,...,2.944920,2.048130,3.336435,2.790056,2.880368,1.979787,3.387613,1.912216,2.069847,0.0


In [11]:
# change string columns into numericals, using doomies function
#df = pd.get_dummies(df,columns=["gdo_rezsoc00","gdo_rezsoc05","gdo_rezsoc10"],dtype="float")
df

,pobtot_ajustada,pobreza,pobreza_e,pobreza_m,vul_car,vul_ing,npnv,ic_rezedu,ic_asalud,ic_cv,...,cpic_asalud,cpic_segsoc,cpic_cv,cpic_sbv,cpic_ali,cpcarencias,cpcarencias3,cpplb,cpplbm,secure_live
0,794304,30.531104,2.264478,28.266627,27.983320,8.419106,33.066469,14.970553,24.034493,4.721187,...,2.327308,2.039859,2.642070,2.785471,2.350738,1.818455,3.311549,1.495813,1.814550,1.0
1,48592,67.111172,8.040704,59.070468,22.439389,5.557604,4.891835,21.222712,15.514032,11.062247,...,2.551830,1.946811,3.043817,3.020979,2.667471,1.887565,3.382552,1.790992,1.945454,0.0
2,53104,61.360527,7.241238,54.119289,29.428583,2.921336,6.289554,27.361207,20.812551,9.880071,...,2.438658,1.923021,3.081348,3.001547,2.847499,1.850954,3.317859,1.818049,1.976999,0.0
3,14101,52.800458,4.769001,48.031458,27.128568,7.709276,12.361698,20.889023,14.071657,11.443449,...,2.495737,1.796453,2.932659,2.960497,2.788465,1.713273,3.355825,1.541247,1.728584,0.0
4,101379,45.338512,6.084037,39.254475,26.262912,8.279864,20.118712,20.578144,16.567818,12.936542,...,2.390123,2.023067,2.764593,3.375908,2.564959,1.852616,3.320841,1.652841,1.993144,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,21016,74.848837,12.301183,62.547654,19.229856,3.177689,2.743618,27.350040,36.056322,5.679670,...,2.730348,2.177080,3.615010,3.326316,2.729491,2.091074,3.403502,2.029167,2.120813,0.0
2452,27385,65.450191,10.203506,55.246687,23.623556,5.007426,5.918827,29.914879,53.313420,2.857021,...,2.522609,2.295629,3.713783,3.069458,3.148968,2.185606,3.334456,2.074749,2.210380,0.0
2453,117528,29.541959,3.535624,26.006335,16.644262,8.828019,44.985759,11.936088,18.316528,2.417258,...,2.562583,2.132257,3.146202,2.999603,2.407763,1.877738,3.363973,1.536344,2.010299,1.0
2454,20456,78.374962,14.607016,63.767946,13.750759,4.440331,3.433948,26.649950,11.769479,10.705211,...,2.944920,2.048130,3.336435,2.790056,2.880368,1.979787,3.387613,1.912216,2.069847,0.0


In [12]:
columnas_numericas = df.select_dtypes(include=['int', 'float','object']).columns
df[columnas_numericas] = df[columnas_numericas].astype(float)
df.dtypes

pobtot_ajustada    float64
pobreza            float64
pobreza_e          float64
pobreza_m          float64
vul_car            float64
vul_ing            float64
npnv               float64
ic_rezedu          float64
ic_asalud          float64
ic_cv              float64
ic_sbv             float64
ic_ali             float64
carencias          float64
carencias3         float64
plb                float64
plb_m              float64
rankin_p           float64
rankin_pe          float64
N_pobreza          float64
N_pobreza_e        float64
N_pobreza_m        float64
N_vul_car          float64
N_vul_ing          float64
N_npnv             float64
N_ic_rezedu        float64
N_ic_asalud        float64
N_ic_segsoc        float64
N_ic_cv            float64
N_ic_sbv           float64
N_ic_ali           float64
N_carencias        float64
N_carencias3       float64
N_plb              float64
N_plb_m            float64
perrankin_p        float64
perrankin_pe       float64
cppobreza          float64
c

In [13]:
#Deviding the dataframe, observations and target
X = df.drop(columns=['secure_live'])
Y = df['secure_live']

#Splitting betwwen train and validation df
X_train, X_val = train_test_split(X,test_size=0.2,random_state=42)
y_train, y_val = train_test_split(Y,test_size=0.2,random_state=42)

In [14]:
columnas_no_numericas = []

for columna in X_val.columns:
    try:
        pd.to_numeric(df[columna])
    except (ValueError, TypeError):
        # Si la conversión falla, la columna no es numérica
        columnas_no_numericas.append(columna)

if columnas_no_numericas:
    print("Se encontraron columnas no numéricas en el DataFrame:")
    print(columnas_no_numericas)
else:
    print("Todas las columnas son numéricas.")


Todas las columnas son numéricas.


In [15]:

def euclidean_distance(x1, x2):
    distance = np.sqrt(np.sum((x1 - x2)**2))
    return distance

class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions) #predictions

    def _predict(self, x):
        # compute the distance
        distances = [euclidean_distance(x, x_train) for x_train in self.X_train]

        # get the closest k
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]

        # majority voye
        most_common = Counter(k_nearest_labels).most_common()
        return most_common[0][0]

In [16]:
# Initialize the KNN classifier with the value K wanted
knn = KNN(k=5)

# Fit the KNN classifier with the data
knn.fit(X_train, y_train)

# Make a prediction for the input data
prediction = knn.predict(X_val)

print(prediction)
# Show the predicted group for the input data
if prediction[0] == 1:
    print("The input data is predicted to belong to the 'Buy' group.")
else:
    print("The input data is predicted to belong to the 'Do Not Buy' group.")

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [21]:
#KNN Implementation using scikit-learn
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train.values, y_train.values)
pred = neigh.predict_proba(X_val.values)

accuracy = accuracy_score(y_val,pred)
accuracy

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import Perceptron
clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(X_train,y_train)
Perceptron()
pred = clf.predict(X_val)
accuracy = accuracy_score(y_val,pred)
accuracy

0.9857723577235772